In [ ]:


# 假设您已经有了模型的实例
model = Transparent_Signal_Processing_Network(signal_processing_modules, feature_extractor_modules, args)

# 加载模型参数
checkpoint = torch.load('path_to_your_saved_checkpoint.ckpt', map_location=torch.device('cpu'))
model.load_state_dict(checkpoint['state_dict'])

# 确保模型在评估模式下
model.eval()
